In [2]:
#firsr of all import all package and load the file
import string
from math import log
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict, Counter
from nltk.corpus import stopwords    
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import json
import nltk
#nltk.download('averaged_perceptron_tagger')
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
jar = 'stanford-ner.jar'
model = 'english.conll.4class.distsim.crf.ser.gz'
model1 = 'english.all.3class.distsim.crf.ser.gz'
model2 = 'english.muc.7class.distsim.crf.ser.gz'
punc = string.punctuation
stopwordsPart = set(stopwords.words('english'))
stopwordsPart.remove('the')  
stopwordsPart.remove('of') 
stopwordsAll = set(stopwords.words('english'))
def opne_json(text):
    with open(text,'r') as input_file:
        document = json.load(input_file)
    return document



def get_tag_model(model,jar):
    return StanfordNERTagger(model,jar)

person_model = get_tag_model(model,jar)
person_model2 = get_tag_model(model1,jar)
number_model = get_tag_model(model2,jar)




documents_dict = opne_json("documents.json")
test_dict = opne_json("testing.json")
dev_dict = opne_json("devel.json")
train_dict = opne_json("training.json")
query_lables = opne_json("QuestionLabel.json")

G:\Anaconda\lib\site-packages\nltk\tag\stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)
G:\Anaconda\lib\site-packages\nltk\tag\stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)
G:\Anaconda\lib\site-packages\nltk\tag\stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


In [1]:
import gensim
import spacy
from scipy import spatial
import json
import numpy as np
import nltk
from gensim.models import Word2Vec, Doc2Vec
from gensim import models
import string
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from collections import defaultdict
from math import log
from textblob import TextBlob
from collections import Counter
from nltk.tag import StanfordNERTagger

%matplotlib inline

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import urllib
import sys
import os
import zipfile
import tarfile
 
import hashlib
import re
import itertools
from sklearn import metrics

G:\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


ImportError: No module named spacy

In [3]:
def preprocess_sentence(sentence):
    words = sentence.split(' ')
    words = [lemmatize(removePunc(x.lower())) for x in words]
    return ' '.join(words)

def named_entity_recognize(sentence,valid_ner):
    #do not preprocess sentence when getting named entity using spacy
    """
    """
    entities = []
    ner_tag = []
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(sentence)
    if valid_ner == 'disable':
        ner_tag = ''
        for ent in doc.ents:
            ner_tag=ent.label_
        if ner_tag == '':
            ner_tag = 'O'
        return ner_tag
    for ent in doc.ents:
        if ent.label_ in valid_ner:
            entities.append(ent)
            ner_tag.append(ent.label_)
    # case 'O'
    if valid_ner == 'O':
        sentence = preprocess_sentence(sentence)
        ents = sentence.split(' ')
        
        """
        add more selection method
        """
        entities += ents
        ner_tag.append('O')
        if entities == []:
            return 'unknown'
    return entities


In [4]:
#get paragraph from the training data
def get_paragraph(docid,documents_dict):
    #get the paragraph that contains the answer
    for i in documents_dict:
        if i['docid'] == docid:
            document = i['text']
            break
    return document




In [5]:
#get TF 
def term_freqs(document):
    tfs = defaultdict(dict)
    tfs_forward = defaultdict(dict)
    doc_id = 0
    for sentence in document:
        for token in word_tokenize(sentence):
            if token not in stopwordsAll and token not in punc:  
                term = lemmatizer.lemmatize(token.lower())
                tfs[term][doc_id] = tfs[term].get(doc_id, 0) + 1 
                tfs_forward[doc_id][term] = tfs[doc_id].get(term, 0) + 1 
        doc_id += 1
    return tfs,doc_id+1,tfs_forward




In [6]:
#build TF_IDF model
def get_tfidf(tfs, total_docment,tfs_forward):
    document_length = {}
    for doc_id,doc_list in tfs_forward.items():
        length = 0
        for term, freq in doc_list.items():
            length += freq ** 2
        length = length **0.5
        document_length[doc_id] =  length
    tfidf = defaultdict(dict)
    for term, doc_list in tfs.items():
        df = len(doc_list)
        for doc_id, freq in doc_list.items(): 
            tfidf[term][doc_id] = (float(tfs[term][doc_id]) * log(total_docment / df))# / document_length[doc_id]
    return tfidf



In [7]:
#find top_k paragraph that may contain the answer
def get_top_k_document(tfidf,query,k,document):
    top_document_id = Counter()
    for token in word_tokenize(query):
        if token not in stopwordsAll:  
             term = lemmatizer.lemmatize(token.lower())
             term_tfidf = tfidf[term]
             for docid, weight in term_tfidf.items():
                 top_document_id[docid] += weight
    top_document_id = top_document_id.most_common(k)
    top_document = []
    for document_id,weight in top_document_id:
        top_document.append(document[document_id])
    return top_document


In [8]:
#filter the key words in query
def get_open_class_word(query):
    tagged = nltk.pos_tag(word_tokenize(query), tagset="universal")
    return [p[0] for p in tagged if (p[1] in ["NOUN","VERB"] or p[0].isdigit()) and p[0] not in ["did","do","was","were"]]

In [9]:
#combine the NER with same tag
def same_tag(ner_output):
    word,tag = '',''
    combo = []
    for word1,tag1 in ner_output:
        if tag1 == tag:
            if word[-1] in ['(',')']:
                word += word1
            if word1 in [')']:
                 word += word1
            else:     
                word += " " + word1
        else:
            combo.append((word,tag))
            tag = tag1
            word = word1
            continue
    if len(combo) != 0:
        combo.pop(0)
    return combo

In [10]:
def most_in(key_words,sentence):
    all_in = True
    len1 = len(key_words)
    word_in = 0
    for i in key_words:
        try:
            index = sentence.index(i)
            word_in += 1 
        except ValueError:
            continue
    return len1 < 2*word_in

def in_key_words(word,key_words):
    in_key = False
    for i in key_words:
        if word.find(i) != -1:       
            in_key = True
            break
    return in_key
            

In [19]:
'''
def get_sentence(train_dict,documents_dict):
    #get the paragraph that contains the answer
    document = []
    docid = 379
    answer_paragraph = 15
    answer_sentence = documents_dict[docid]['text'][answer_paragraph]
       
    return answer_sentence
answer_sentence = get_sentence(train_dict,documents_dict)
print answer_sentence
'''

Beyoncé further expanded her acting career, starring as blues singer Etta James in the 2008 musical biopic, Cadillac Records. Her performance in the film received praise from critics, and she garnered several nominations for her portrayal of James, including a Satellite Award nomination for Best Supporting Actress, and a NAACP Image Award nomination for Outstanding Supporting Actress. Beyoncé donated her entire salary from the film to Phoenix House, an organization of rehabilitation centers for heroin addicts around the country. On January 20, 2009, Beyoncé performed James' "At Last" at the First Couple's first inaugural ball. Beyoncé starred opposite Ali Larter and Idris Elba in the thriller, Obsessed. She played Sharon Charles, a mother and wife who learns of a woman's obsessive behavior over her husband. Although the film received negative reviews from critics, the movie did well at the US box office, grossing $68 million—$60 million more than Cadillac Records—on a budget of $20 mil

In [11]:
document = get_paragraph(409,documents_dict)
tfs,total_docment,tfs_forward = term_freqs(document)
tfidf = get_tfidf(tfs, total_docment,tfs_forward)


In [12]:
query ="Whose performance in Anne of the Thousand Days garnered a Best Actor nod?"
top_k = get_top_k_document(tfidf,query,5,document)
print top_k

[u"At this time, Hal B. Wallis, who had latterly worked as a major producer at Paramount, moved over to Universal, where he produced several films, among them a lavish version of Maxwell Anderson's Anne of the Thousand Days (1969), and the equally lavish Mary, Queen of Scots (1971). Though neither could claim to be a big financial hit, both films received Academy Award nominations, and Anne was nominated for Best Picture, Best Actor (Richard Burton), Best Actress (Genevi\xe8ve Bujold), and Best Supporting Actor (Anthony Quayle). Wallis retired from Universal after making the film Rooster Cogburn (1975), a sequel to True Grit (1969), which Wallis had produced at Paramount. Rooster Cogburn co-starred John Wayne, reprising his Oscar-winning role from the earlier film, and Katharine Hepburn, their only film together. The film was only a moderate success.", u"The production arm of the studio still struggled. While there were to be a few hits like The Killers (1946) and The Naked City (1948)

In [13]:

def get_answer_list(query,answer_type):
    key_words = get_open_class_word(query)
    print key_words
    answer_list = {}
    answer_type = answer_type
    for ans_sentence in top_k:
        if most_in(key_words,ans_sentence) == False:
            continue
        word_list =  []
        for word in word_tokenize(ans_sentence):
            word_list.append(word)    
        word_list_tag = number_model.tag(word_list)
        word_list_tag = same_tag(word_list_tag)
        for word,tags in word_list_tag:
            if word not in answer_list.keys():
                if word not in stopwordsAll and word not in punc and tags == answer_type and word not in key_words and in_key_words(word,key_words) == False:
                    distance_list = []
                    distance = 0
                    for key_word in key_words:
                        try:
                            index = ans_sentence.index(key_word)
                            distance_list.append(index)
                        except ValueError:
                            distance_list.append(5000)
                    for index in distance_list:
                        try:
                            distance += abs(index - ans_sentence.index(word))
                        except ValueError:
                            print '2'
                    answer_list[word] = distance
    if  len(answer_list.items()) != 0:
        return sorted(answer_list.items(), lambda x, y: cmp(x[1], y[1]))#[0][0].lower()

In [14]:

print get_answer_list(query,"PERSON")


['performance', 'Anne', 'Thousand', 'Days', 'garnered', 'Best', 'Actor', 'nod']
[(u'Richard Burton', 14415), (u'Genevi\xe8ve Bujold)', 14477), (u'Anthony Quayle', 14569), (u'Mary', 14715), (u'John Wayne', 14949), (u'Maxwell Anderson', 15047), (u'Katharine Hepburn', 15093), (u'Wallis', 16287), (u'Hal B. Wallis', 16343)]


In [17]:
import csv
def output(labels):
    qa_id = 0
    csv_file = open('output.csv', 'w')
    writer = csv.writer(csv_file)
    writer.writerow(['id', 'answer'])
    for i in labels:
        query =  i['question_answer_type_dict'].keys()[0]
        answer_type = i['question_answer_type_dict'].values()[0]
        answer = get_answer_list(query,get_top_k_document(tfidf,query,10,document),answer_type)
        print answer
        writer.writerow([qa_id, answer])
        qa_id += 1
    csv_file.close()
'''
    for article in data:
        collection = article['sentences']
        tf, total_docs = get_term_frequencies(collection)
        #tfidf = get_tfidf(tf, total_docs)
        tagged_sentences = rechunk(entity_recognize(collection))       
        for qa in article['qa']:
            question = qa['question']
            qa_id = qa['id']
            okapibm25 = get_okapibm25(tf, total_docs, collection)
            sentence_id = retrieve_sentence(okapibm25, question)
            if len(sentence_id) == 0:
                writer.writerow([qa_id, 'Not sure'])
            else:
                doc_id = sentence_id[0][0]
                first_pass_result = first_pass(tagged_sentences[doc_id], question, collection[doc_id])
                if first_pass_result == []:
                    writer.writerow([qa_id, 'Not sure'])
                else:
                    second_pass_result = second_pass(first_pass_result, question)
                    third_pass_result = third_pass(question, second_pass_result)
                    writer.writerow([qa_id, third_pass_result])
'''
    

output(query_lables)

TypeError: get_answer_list() takes exactly 2 arguments (3 given)

In [24]:
def get_accuracy(train_dict,documents_dict):
    total = 0
    right = 0
    docidstart = train_dict[0]['docid']
    document = get_paragraph(docidstart,documents_dict)
    tfs,total_docment,tfs_forward = term_freqs(document)
    tfidf = get_tfidf(tfs, total_docment,tfs_forward)
    for i in train_dict:
        docid = i['docid']
        if docid != docidstart:
            print docid
            document = get_paragraph(docid,documents_dict)
            tfs,total_docment,tfs_forward = term_freqs(document)
            tfidf = get_tfidf(tfs, total_docment,tfs_forward)
            docidstart = docid
        query = i['question']
        answer_paragraph = i['answer_paragraph']
        answer_sentence = documents_dict[docid]['text'][answer_paragraph]
        top_k = get_top_k_document(tfidf,query,5,document)
        if answer_sentence in top_k:
            right += 1
        total += 1
    print right
    print total
    return float(right)/total

get_accuracy(dev_dict,documents_dict)

381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
2856
3097


0.9221827575072651

In [ ]:
def get_answer_list(query):
    key_words = get_open_class_word(query)
    print key_words
    answer_list = {}
    answer_type = answer_type
    for ans_sentence in top_k:
        if most_in(key_words,ans_sentence) == False:
            continue
        word_list_tag = named_entity_recognize(ans_sentence,answer_type)
        for word,tags in word_list_tag:
            if word not in answer_list.keys():
                if word not in stopwordsAll and word not in punc and tags == answer_type and word not in key_words and in_key_words(word,key_words) == False:
                    distance_list = []
                    distance = 0
                    for key_word in key_words:
                        try:
                            index = ans_sentence.index(key_word)
                            distance_list.append(index)
                        except ValueError:
                            distance_list.append(5000)
                    for index in distance_list:
                        try:
                            distance += abs(index - ans_sentence.index(word))
                        except ValueError:
                            print '2'
                    answer_list[word] = distance
    if  len(answer_list.items()) != 0:
        return sorted(answer_list.items(), lambda x, y: cmp(x[1], y[1]))#[0][0].lower()

In [ ]:
print get_answer_list(query,"PERSON")